# 구문 분석: Part I - formal grammar 
---
All rights reserved, 2021,2022 By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - https://www.nltk.org/book/ch08.html
    - nltk book 8.Analyzing sentence structure() 내용 참고. 
        - 8장 예제 인용.
- 참고 서적
    - Natural Language Processing with Python Cookbook
        - Krishna Bhavsar, Naresh Kumar, Pratap Dangeti, Packt Publishing, 2017.
    - Chapter 6, Chapter 7의 예제 일부 사용.

여기서 다룰 내용은 다음과 같습니다.
- 다양한 문장 구조를 표현하려면 형식 문법을 어떻게 사용하면 될까? 
- 구문 트리를 사용하여 문장 구조를 어떻게 보여줄 수 있을까?
- 파서가 문장을 어떻게 분석하고, 구문 트리를 자동으로 만들어 낼까?

In [97]:
import nltk, re
import sys, string

## 1.   형식 문법(formal grammar)

### 1.1  문법과 생성 규칙

언어란 (문법에 맞는) 문장의 방대한 집합입니다. 
- **형식 문법**은 문장을 생성하는 규칙을 정의합니다.
- 형식 문법은 이 집합의 원소인 문장을 **생성(generate)** 하는 형식을 갖춘 표기법입니다.
    - 문법은 대부분 **S → S and S** 와 같은 순환 생성 규칙으로 이루어집니다.
    - 문법에 속한 각 부분의 의미 분석을 통해 문장의 의미를 유추할 수 있습니다.

간단한 문법 예제를 통해 문법이 문장을 어떻게 생성할 수 있는지 알아보겠습니다.
- 생성 규칙(prouction)은 문자열(*str*) 리스트로 표현. 
    - ' '은 빈 문자열(empty string)을 가리킴. 
    - 빈 문자열은 길이가 0인 문자열(즉 생략 가능).

In [98]:
productions = [ # python list
    "ROOT -> WORD",
    "WORD -> ' '",  # empty string
    "WORD -> NUMBER LETTER",
    "WORD -> LETTER"
]

- 단계 1: join()을 사용하여 모든 생성 규칙을 하나의 문자열(*grammarString*)로 합침.
- 단계 2: 문법(*gr*)을 생성하기 위해 **nltk.CFG.fromstring()** 호출.
- 각각의 type을 눈여겨 보기 바랍니다.

In [99]:
grammarString = "\n".join(productions)
gr = nltk.CFG.fromstring(grammarString)
print(type(productions), type(grammarString), type(gr))

<class 'list'> <class 'str'> <class 'nltk.grammar.CFG'>


- 문법 gr은 생성규칙이 4개이며, start symbol은 ROOT.
    - gr.start() : start symbol of the grammar

In [100]:
print(gr) 
print(gr.start()) 

Grammar with 4 productions (start state = ROOT)
    ROOT -> WORD
    WORD -> ' '
    WORD -> NUMBER LETTER
    WORD -> LETTER
ROOT


- gr.productions()\[1\].lhs(): lhs-left hand side(lhs) of the 2nd production
- gr.productions()\[2\].rhs(): rhs-right hand side(rhs) of the 3rd production

In [101]:
print(gr.productions())
print(gr.productions()[1].lhs()) 
print(gr.productions()[2].rhs()) 

[ROOT -> WORD, WORD -> ' ', WORD -> NUMBER LETTER, WORD -> LETTER]
WORD
(NUMBER, LETTER)


문법의 4가지 구성 요소 중 아직 terminal이 없습니다. 문장(sentence)은 terminal 기호로만 이루어기 때문에 terminal을 추가해야 합니다. 숫자 3개(0,1,2)와 문자 3개(a, b, c)를 terminal로 추가해 보겠습니다.

In [102]:
print(string.digits)
print(string.ascii_lowercase)

0123456789
abcdefghijklmnopqrstuvwxyz


- **NUMBER, LETTER**가 terminal을 생성하도록 하면 됩니다. 
    - 문자 상수, 숫자 상수는 따옴표(apstrophe)로 묶어야 합니다. 
        - 이를 *literal* 이라고 부릅니다.
    - 생성 규칙의 lhs 기호가 같을 경우 
        - '|'(vertical bar)를 사용하여 묶을 수 있습니다.

In [103]:
digits = list(string.digits)

#숫자 상수 0, 1, 2를 NUMBER 의 생성규칙에 추가
#상수는 따옴표('')로 묶어야 함.
for digit in digits[:3]:
    productions.append("NUMBER -> '{w}'".format(w=digit)) 
productions[-3:]    

["NUMBER -> '0'", "NUMBER -> '1'", "NUMBER -> '2'"]


'|'를 사용해 lhs가 같다면 여러 개 생성규칙을 함께 묶을 수 있습니다.

In [104]:
letters = "' | '".join(list(string.ascii_lowercase[:3]))    
productions.append("LETTER -> '{w}'".format(w=letters)) 
productions[-3:]  

["NUMBER -> '1'", "NUMBER -> '2'", "LETTER -> 'a' | 'b' | 'c'"]

- 처음에 4개의 규칙만 있었지만, 
    - **NUMBER**와 **LETTER**에 관한 규칙을 각각 3개씩 추가해 
    - 모두 10개의 규칙을 갖게 되었습니다.

In [105]:
productions

['ROOT -> WORD',
 "WORD -> ' '",
 'WORD -> NUMBER LETTER',
 'WORD -> LETTER',
 "NUMBER -> '0'",
 "NUMBER -> '1'",
 "NUMBER -> '2'",
 "LETTER -> 'a' | 'b' | 'c'"]

- 문법은 구문 분석 과정에서 구문 트리로 표현할 수 있습니다.
- 구문 트리는 한 개의 root를 갖습니다. 
    - 이 문법의 root는 첫 번째 규칙의 lhs인 **ROOT**입니다.

In [106]:
grammarRules = "\n".join(productions)
gr = nltk.CFG.fromstring(grammarRules)
print(gr)

Grammar with 10 productions (start state = ROOT)
    ROOT -> WORD
    WORD -> ' '
    WORD -> NUMBER LETTER
    WORD -> LETTER
    NUMBER -> '0'
    NUMBER -> '1'
    NUMBER -> '2'
    LETTER -> 'a'
    LETTER -> 'b'
    LETTER -> 'c'


### 1.2  문장 생성(generate) 

구문 분석기(**RecursiveDescentParser**)를 실행하면 구문 트리가 만들어집니다.
- 구문 분석을 하기 위한 입력으로 ['0', 'a']를 전달했습니다. 
    - 여기서 '0', 'a'는 문법에서 정의한 **token**입니다.
        - '0, a'와 같이 하나의 문자열로 입력하면 syntax error가 발생합니다. 
        - 왜 그럴까요? 에러 메시지를 보면서 확인해 보세요.
- **tree.draw()**는 새 창(window)에 구문 트리를 출력합니다.
    - tree를 print하면 중첩된 괄호 표기법으로 트리의 내용을 출력합니다.
    - 창을 보고 나서 닫아야만 \*(실행 중) 표시가 없어지고 셀 번호가 표시됩니다.
- 트리에 관한 상세한 설명은 **1.4절** 참고

In [107]:
parser = nltk.RecursiveDescentParser(gr)
for tree in parser.parse(['0', 'a']):
#for tree in parser.parse(['0, a']):
    print(tree)
    tree.draw()

(ROOT (WORD (NUMBER 0) (LETTER a)))


- 트리의 길이(depth)는 4입니다. 
- root에서 leaf까지 노드가 모두 4개이기 때문이죠.

In [108]:
print('depth of tree =', tree.height())

depth of tree = 4


주어진 문법이 어떤 문장을 생성하는지 모두 찾아보겠습니다. 
- 문장은 terminal 기호로만 구성되어야 합니다.
- 생성 범위를 제한하기 위해 `depth`와 `n`을 지정합니다.
    - **depth**는 구문 트리의 `depth`, **n**은 생성할 문장의 최대 갯수입니다.
    - n을 15로 지정했지만 생성 가능한 문장 개수는 13개이며, 
        - 구문트리의 depth는 4입니다.

In [109]:
from nltk.parse.generate import generate

i = 0
for sent in generate(gr, n=15, depth=4):  
    sents = "".join(sent)
    print("{} >>> generated word: {}, size: {}".format(i, sents, len(sents)))   
    i += 1

0 >>> generated word:  , size: 1
1 >>> generated word: 0a, size: 2
2 >>> generated word: 0b, size: 2
3 >>> generated word: 0c, size: 2
4 >>> generated word: 1a, size: 2
5 >>> generated word: 1b, size: 2
6 >>> generated word: 1c, size: 2
7 >>> generated word: 2a, size: 2
8 >>> generated word: 2b, size: 2
9 >>> generated word: 2c, size: 2
10 >>> generated word: a, size: 1
11 >>> generated word: b, size: 1
12 >>> generated word: c, size: 1


### Quiz 1

- 슬라이드 7(강의노트(ch5-1))의 문법을 nltk의 fromstring()을 사용해 정의
    - 단, digit는 4개 숫자(0, 1, 2, 3)로 제한.
- RecursiveDescentParser와 generate()를 사용하여 문장 생성
    - 이 문법이 생성할 수 있는  모든 문장을 생성할 것.

In [123]:
# quiz 1에 대한 코드를 추가하세요

#생성 규칙 만들기 
productions = [ 
    "list -> list '+' digit",
    "list -> list '-' digit",
    "list -> digit"
]


#digit 4개의 숫자 (0,1,2,3) 추가하기 
digits = list(string.digits)
for digit in digits[:4]:
    productions.append("digit -> '{w}'".format(w=digit))
    
    
grammarString = "\n".join(productions)
gr = nltk.CFG.fromstring(grammarString)
i = 0
#만든 생성 규칙에 맞는 문장 20개 출력 
for sent in generate(gr, n=20, depth = 5): 
    sents = "".join(sent)
    print("{}. >>> generated word: {}, size: {}".format(i+1, sents, len(sents)))   
    i += 1

1. >>> generated word: 0+0+0, size: 5
2. >>> generated word: 0+0+1, size: 5
3. >>> generated word: 0+0+2, size: 5
4. >>> generated word: 0+0+3, size: 5
5. >>> generated word: 0+1+0, size: 5
6. >>> generated word: 0+1+1, size: 5
7. >>> generated word: 0+1+2, size: 5
8. >>> generated word: 0+1+3, size: 5
9. >>> generated word: 0+2+0, size: 5
10. >>> generated word: 0+2+1, size: 5
11. >>> generated word: 0+2+2, size: 5
12. >>> generated word: 0+2+3, size: 5
13. >>> generated word: 0+3+0, size: 5
14. >>> generated word: 0+3+1, size: 5
15. >>> generated word: 0+3+2, size: 5
16. >>> generated word: 0+3+3, size: 5
17. >>> generated word: 1+0+0, size: 5
18. >>> generated word: 1+0+1, size: 5
19. >>> generated word: 1+0+2, size: 5
20. >>> generated word: 1+0+3, size: 5


### 1.3   모호(Ambiguity) 

형식 문법에서 가장 다루기 힘든 것은 모호한 문장을 분석할 때입니다. 문장이 모호(ambiguity)하다는 게 뭘까요? 형식 언어는 dangling-else 문제처럼 제한된 경우에만 모호함이 발생하지만, 자연 언어는 언제든 다양한 형태로 모호함이 발생할 수 있습니다. 문장 구조를 제대로 파악하지 못하면 문장 의미도 엉뚱하게 해석될 수 밖에 없습니다. 아래 문장은 Groucho Mark가 주인공으로 나온 영화 'Animal Cracker'(1930)에서 나온 대사입니다. 그래서 문법 이름이 grocho_grammar 입니다.
- "I shot an elephant in my pajamas." 
- 이 문장의 모호함에 대해 살펴보겠습니다. 
    - 나는 잠옷에 그려진 코끼리를 (카메라로) 찍었다. 
        - 또는 잠옷에 그려진 코끼리를 (총으로) 쐈다. (코미디 영화이기 때문에 가능한 해석)
    - 나는 파자마를 입은 채 코끼리를 (카메라로) 찍었다.

In [16]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

문장을 구문 분석할 경우 2가지 트리가 생성됩니다. 
- 구문 분석기로 ChartParser를 사용합니다. 
    - 참고로 ChartParser가 RecursiveDescentParser보다는 성능이 좋습니다.
    - 구문 분석기에 관한 상세 내용은 다음 실습 파일에서 제공됩니다.
- "in my pajamas"란 전치사구가 사진 찍는 상태를 설명(첫 번째 트리) 
- "in my pajamas"란 전치사구가 코끼리를 수식하기 위해 사용(두 번째 트리)
- 어떤 문장에 대해 2가지 구문 트리(syntax tree)를 생성한다는 것은 
    - 문법이 모호함을 의미합니다. 

- 아래 예제에서 품사 약어는 다음과 같습니다.
    - *NP*(명사구), *Adj*(형용사, adjective), *Conj*(접속사, conjuction), 
    - *N*(명사, noun), *PP*(부사구), *V*(동사, verb)

In [17]:
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
    print(tree)
    #tree.draw()

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


**어휘 모호성(lexical ambiguity)** 
- 단어가 중의적 의미(double meaning)를 갖고 있는 경우
    - *I went to the bank.* 
- **bank**가 강둑(river bank) 또는 은행 중 어떤 의미인지는 
- 이어지는 문장을 살펴봐야 합니다.

**형용사의 수식 대상이 모호**
- 형용사의 수식 범위가 어디까지인지 명확하지 않은 경우입니다.
- **old men and women**은 아래처럼 다르게 해석할 수 있기 때문입니다.
    - a. old (men and women)
    - b. (old men) and women
- 각각에 대해 구문 트리를 출력하면 보다 명확히 확인할 수 있습니다.

In [18]:
from nltk import Tree

In [19]:
s11 = '(NP (Adj old) (NP (N men) (Conj and) (N women)))'
tree11 = Tree.fromstring(s11)
tree11.draw()  

In [20]:
s12 = '(NP (NP (Adj old) (N men)) (Conj and) (NP (N women)))'
tree12 = Tree.fromstring(s12)
tree12.draw()  

**전치사구(prepositional phrases, PP) 해석이 모호**
- **I saw the man with a telescope.** - 누가 망원경을 갖고 있을까? 
- 조금 더 분명히 하기 위해 아래 2개의 문장을 살펴보겠습니다.
    - (괄호 안 내용은 문장을 부연 설명)
    - a.The policeman saw a burglar with a gun. 
        - (not some other burglar)
    - b.The policeman saw a burglar with a telescope. 
        - (not with his naked eye)
- 2개 문장 모두 전치사구(**PP**)를 갖고 있습니다. 
- 위 주석대로 문장을 해석한다면 
    - (a)에서는 PP가 명사 burglar를 수식하며, 
    - (b)에서는 PP가 동사 saw를 수식합니다.  

In [21]:
s21 = '(S (NP the policeman) (VP (V saw) (NP (NP the burglar) (PP with a gun))))'
s22 = '(S (NP the policeman) (VP (V saw) (NP the burglar) (PP with a telescope)))'

In [22]:
tree21 = Tree.fromstring(s21)
tree21.draw()
tree22 = Tree.fromstring(s22)
tree22.draw()

In [23]:
tree21.leaves()

['the', 'policeman', 'saw', 'the', 'burglar', 'with', 'a', 'gun']

- tree의 leaf 노드는 왼쪽에서 오른쪽 방향으로 리스트에 저장되어 있습니다. 
- 위 2개 문장은 마지막 단어(gun, telescope)만 다릅니다. 

In [24]:
print(tree21.leaves()[-1], tree22.leaves()[-1])
tree21.leaves()[:-1] == tree22.leaves()[:-1]

gun telescope


True

- 2개 문장은 **전치사구(PP)** 가 수식하는 대상이 다르기 때문에, 
- 문장 구조가 다르며, 따라서 구문 트리도 다릅니다.

In [25]:
print(tree21.height(), tree22.height())
tree21.height() == tree22.height()

5 4


False

### 1.4 nltk에서 Tree 사용
구문 분석 과정은 구문 트리로 시각화해서 나타낼 수 있습니다. 

Tree를 사용하려면 **from nltk import Tree** 문장을 선언해야 합니다.

In [26]:
from nltk import Tree

괄호 표현에서 왼쪽 기호는 parent, 오른쪽 리스트는 child 가 됩니다.

In [27]:
tree = nltk.Tree('NP', ['John'])
print("height={}, leaves={}".format(tree.height(), tree.leaves()))
print(tree)
#tree.draw()

height=2, leaves=['John']
(NP John)


In [28]:
tree2 = nltk.Tree('NP', ['the', 'man'])
print(tree2)
#tree2.draw()

(NP the man)


이미 만들어진 tree를 새롭게 만드는 tree의 child로 포함시킬 수 있습니다.

In [29]:
tree3 = nltk.Tree('VP', ['saw', tree2])
print(tree3)
tree3.draw()

(VP saw (NP the man))


In [30]:
tree4 = nltk.Tree('S', [tree, tree3])
print(tree4)
tree4.draw()

(S (NP John) (VP saw (NP the man)))


아래 코드를 실행하기 전에 tree4의 parse tree를 확인하기 바랍니다.

In [31]:
print(tree4[1]) #root 노드의 2번째 child를 출력

(VP saw (NP the man))


In [32]:
print(tree4[1].label) #root 노드의 2번째 child를 출력

<bound method Tree.label of Tree('VP', ['saw', Tree('NP', ['the', 'man'])])>


In [33]:
tree4.leaves() #tree4의 모든 leaf node를 출력

['John', 'saw', 'the', 'man']

In [34]:
tree4[1,1,1] #root의 2번째(오른쪽) child의 2번째(오른쪽 child)를 출력

'man'

In [35]:
#tree4[0,0]
tree4[1,0]

'saw'